In [ ]:
import pandas as pd
import seaborn as sns
from glob import glob
sns.set_theme(style="ticks", font="Linux Biolinum O", font_scale=1.2)

frames = [pd.read_csv(data) for data in glob('../data/operation-*.txt')]
for frame in frames:
    if 'num_participant' not in frame:
        frame['num_participant'] = 10000
data = pd.concat(frames, ignore_index=True)
data = data[data['comment'].isna()]

data = data[data['num_participant'] == 10000]
data = data[(data['inner_k'] == 32) & (data['inner_n'] == 80) & (data['outer_k'] == 8) & (data['outer_n'] == 10)]

def protocol_operation(row):
    return f'{row["protocol"].capitalize()} - {row["operation"]}'

data["Protocol - Operation"] = data.apply(protocol_operation, axis=1)
data

In [ ]:
hue_order = ["Entropy - put", "Kademlia - put", "Entropy - get", "Kademlia - get"]
entropy_palette = sns.cubehelix_palette(rot=0., light=0.75, n_colors=4)
kademlia_palette = sns.cubehelix_palette(start=2.5, rot=0., dark=0.25, n_colors=4)
palette = [c for p in zip(entropy_palette, reversed(kademlia_palette)) for c in p]

sns.relplot(data=data[data['operation'] != 'repair'],
    kind="line",
    x="num_concurrent", y="latency", hue="Protocol - Operation",
    hue_order=hue_order, palette=palette[:4],
    markers=['o'] * 4, dashes=False, style='Protocol - Operation',
    height=2.56, aspect=2.) \
.set_xlabels("Concurrent Operation") \
.set_ylabels("Latency (s)") \
.set(ylim=[0, 25]) \
.savefig("../../graphs/concurrent-put-get.pdf")

In [ ]:
hue_order = ["Entropy - repair", "Kademlia - repair"]

sns.relplot(data=data[data['operation'] == 'repair'],
    kind="line",
    x="num_concurrent", y="latency", hue="Protocol - Operation",
    hue_order=hue_order, palette=palette[4:],
    markers=['o'] * 2, dashes=False, style='Protocol - Operation',
    height=2.56, aspect=2.) \
.set_xlabels("Concurrent Operation") \
.set_ylabels("Latency (s)") \
.set(ylim=[0, 3.99]) \
.savefig("../../graphs/concurrent-repair.pdf")